# 5.9 수업 : 동적 HTML 페이지 받기
### 셀레니움 활용해서 할 것 - form 태그 이용해서

In [1]:
from urllib import parse

header = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}
# Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36
def getDownload(url, params={}, retries=3):
    resp = None
    
    try:
        resp = requests.get(url, params=params, headers=header)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and retries > 0:
            print(retries)
            resp = getDownload(url, params, retries-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
            print(e.response.headers)
            
    return resp

In [2]:
import requests
from bs4 import BeautifulSoup

## url을 그냥 가져오는게 아니라 네트워크 확인해서 ajax를 가져와야함

In [3]:
url = "http://example.webscraping.com/places/default/search"
url = "http://example.webscraping.com/places/ajax/search.json"
params = {
    "search_term":"",
    "page_size":10,
    "page":0
}

params["search_term"] = "korea"

html = getDownload(url, params)
# dom = BeautifulSoup(html.content, "lxml")

In [4]:
html.headers["Content-Type"]

'application/json'

### json parsing 해야함

### DOM 활용해서 가져오기

In [5]:
for _ in html.json()["records"]:
    print(_["pretty_link"])
    dom = BeautifulSoup(_["pretty_link"], "lxml")
    print([_["href"] for _ in dom.find_all("a")])
#     break

<div><a href="/places/default/view/North-Korea-165"><img src="/places/static/images/flags/kp.png" /> North Korea</a></div>
['/places/default/view/North-Korea-165']
<div><a href="/places/default/view/South-Korea-211"><img src="/places/static/images/flags/kr.png" /> South Korea</a></div>
['/places/default/view/South-Korea-211']


### 정규식 사용하여 가져오기

In [6]:
import re

text = html.json()["records"][0]["pretty_link"]
group1 = re.findall("<a href=\"(.+)\">(.+)</a>", text)
group2 = re.findall("<a href=\"(.+)\"><img src=\"(.+)\" /> (.+)</a>", text)

print(group2)
requests.compat.urljoin(url, group2[0][0]), \
img_url = requests.compat.urljoin(url, group2[0][1]), group2[0][2]

[('/places/default/view/North-Korea-165', '/places/static/images/flags/kp.png', 'North Korea')]


('http://example.webscraping.com/places/default/view/North-Korea-165',
 'http://example.webscraping.com/places/static/images/flags/kp.png',
 'North Korea')

### 로그인 : NAVER
### (JS)암호화 -> 토큰(AUTH) -> 암호화 -> SSO -> Session 다 따라가야함
### 그래서 셀레니움을 활용할 것임
### 1. 브라우저를 자동화 원래는 테스트 목적
### 2. js를 돌림
### 3. DOM을 받고 활용 가능

In [19]:
from selenium import webdriver

In [59]:
browser = webdriver.Chrome()
# chrome driver 다운해야 함

In [21]:
url = "http://example.webscraping.com/places/default/search"
browser.get(url)

## DOM에 반영시키기

In [30]:
# browser.find_element_by_id("search_term").send_keys("fr")
browser.find_element_by_css_selector("#search_term").clear()

### xpath 활용법
/ -> 자식<br>
// -> 자손<br>
. -> 현재 위치<br>
[속성] => [@키 = 밸류]<br>

In [31]:
browser.find_element_by_xpath("//input[@id='search_term']").send_keys("korea")
browser.find_element_by_id("search").click()

In [34]:
for _ in browser.find_elements_by_xpath("//div[@id='results']//a"):
    print(_.tag_name)
    print(_.text)
    print(_.get_attribute("href"))

a
North Korea
http://example.webscraping.com/places/default/view/North-Korea-165
a
South Korea
http://example.webscraping.com/places/default/view/South-Korea-211


### DOM으로도 가능 - 이게 더 편할 것임

In [40]:
dom = BeautifulSoup(browser.page_source, "lxml")

In [41]:
for _ in dom.select("#results a"):
    print(_.text.strip())
    print(_["href"])

North Korea
/places/default/view/North-Korea-165
South Korea
/places/default/view/South-Korea-211


# NAVER 로그인
## json 파일 만들기

In [52]:
%%writefile account.json
{
    "id":"본인아이디",
    "pw":"본인pw"
}

Overwriting account.json


In [53]:
import json 

with open("./account.json", "r", encoding="utf-8") as fp:
    account = json.load(fp)

In [54]:
account["id"]

'seungho546'

In [65]:
browser = webdriver.Chrome()
url = "https://nid.naver.com/nidlogin.login?url=http%3A%2F%2Fmail.naver.com%2F"
browser.get(url)

In [66]:
browser.find_element_by_xpath("//input[@id='id']").clear()
browser.find_element_by_name("id").send_keys(account["id"])
browser.find_element_by_id("pw").clear()
browser.find_element_by_css_selector("#pw").send_keys(account["pw"])

In [63]:
browser.find_element_by_css_selector("input.btn_global").click()

In [64]:
browser.close()

### 보안설정 확인하고 해야함 일단 로그인 된다는 전제하에 시작

In [72]:
for _ in browser.find_elements_by_css_selector("strong.mail_title"):
    print(_.text)

[네이버페이]주문하신 상품이 발송되었습니다
인터파크에서 구매하신 상품의 리뷰를 작성해주세요.
인터파크 판매자가 상품을 발송완료 처리를 하였습니다.
[AI Center] 비밀번호 초기화
(한국데이터산업진흥원)_SQL, 데이터분석 자격취득자 채용우대 안내
인터파크에서 주문하신 내역입니다.
매일프로그래밍 - 코딩테스트 문제 11
[분석완료] 최승호님 활동으로 시몬스 채용 공고가 매칭되었습니다.
성곡도서관 IEEE Authorship Seminar에 많은 참석 바랍니다.
[인터파크]주문하신 상품의 구매확정이 완료되었습니다
[인공지능 및 기계학습 개론Ⅰ] KOOC 인공지능 및 기계학습 개론 1 학습자 여러분께.
AWS 계정이 확인됨
매일프로그래밍 - 코딩테스트 문제 10
[카카오] 코딩 테스트 완료 안내
[카카오] 2019 하계 추천 시스템 연구 / 개발 인턴 모집 - 1차 코딩테스트 결과 안내
[네이버페이]구매하신 제품을 무사히 받으셨다면, 구매를 확정 부탁드립니다.
[한국IT비즈니스진흥협회]AI이노베이션스퀘어 추가합격자 선정안내
[국제교류팀] 2019학년도 하계 어학연수 프로그램 안내
[카카오/추천팀] 2019년 하계 인턴 지원자 대상 설문
(광고)프로그래머스 개편 안내
Thanks for taking the [카카오/추천팀] 코딩테스트 warm-up - v2
[카카오] 추천 시스템 연구/개발 인턴 공고의 "코딩테스트 상세 시간" 안내드립니다. (1차 코딩 테스트 접근 가능 시간 : 4.27(토) 13:00 ~ 13:10)
[카카오] 추천 시스템 연구/개발 인턴 공고의 "코딩테스트 상세 시간" 안내드립니다. (1차 코딩 테스트 접근 가능 시간 : 4.27(토) 13:00 ~ 13:10)
[카카오] 코딩 테스트 안내
[카카오] 서류전형 합격을 축하합니다!
(광고) 독점공개! 😀 카카오의 블록체인 프로젝트, Klaytn 튜토리얼
인터파크 판매자가 상품을 발송완료 처리를 하였습니다.
[네이버페이]주문하신 상품이 발송되었습니다
2019학년도 하계 해외봉사 프로그램 (글로벌

In [73]:
dom = BeautifulSoup(browser.page_source, "lxml")

In [77]:
for _ in dom.select("strong.mail_title"):
    print(_.text)

메일 제목:[네이버페이]주문하신 상품이 발송되었습니다
메일 제목:인터파크에서 구매하신 상품의 리뷰를 작성해주세요.
메일 제목:인터파크 판매자가 상품을 발송완료 처리를 하였습니다.
메일 제목:[AI Center] 비밀번호 초기화
메일 제목:(한국데이터산업진흥원)_SQL, 데이터분석 자격취득자 채용우대 안내
메일 제목:인터파크에서 주문하신 내역입니다.
메일 제목:매일프로그래밍 - 코딩테스트 문제 11
메일 제목:[분석완료] 최승호님 활동으로 시몬스 채용 공고가 매칭되었습니다.
메일 제목:성곡도서관 IEEE Authorship Seminar에 많은 참석 바랍니다.
메일 제목:[인터파크]주문하신 상품의 구매확정이 완료되었습니다
메일 제목:[인공지능 및 기계학습 개론Ⅰ] KOOC 인공지능 및 기계학습 개론 1 학습자 여러분께.
메일 제목:AWS 계정이 확인됨
메일 제목:매일프로그래밍 - 코딩테스트 문제 10
메일 제목:[카카오] 코딩 테스트 완료 안내
메일 제목:[카카오] 2019 하계 추천 시스템 연구 / 개발 인턴 모집 - 1차 코딩테스트 결과 안내
메일 제목:[네이버페이]구매하신 제품을 무사히 받으셨다면, 구매를 확정 부탁드립니다.
메일 제목:[한국IT비즈니스진흥협회]AI이노베이션스퀘어 추가합격자 선정안내
메일 제목:[국제교류팀] 2019학년도 하계 어학연수 프로그램 안내
메일 제목:[카카오/추천팀] 2019년 하계 인턴 지원자 대상 설문
메일 제목:(광고)프로그래머스 개편 안내
메일 제목:Thanks for taking the [카카오/추천팀] 코딩테스트 warm-up - v2
메일 제목:[카카오] 추천 시스템 연구/개발 인턴 공고의  "코딩테스트 상세 시간" 안내드립니다. (1차 코딩 테스트 접근 가능 시간 : 4.27(토) 13:00 ~ 13:10)
메일 제목:[카카오] 추천 시스템 연구/개발 인턴 공고의  "코딩테스트 상세 시간" 안내드립니다. (1차 코딩 테스트 접근 가능 시간 : 4.27(토) 13:00 ~ 13:10)
메일 제목:[카카오] 코딩 테스트

# 스팸메일함 메일 제목 긁어오기

In [80]:
# 스팸메일함 이동
browser.find_element_by_id("5_fol").click()

In [82]:
for _ in browser.find_elements_by_css_selector("strong.mail_title"):
    print(_.text)

(광고) 창업을 생각중이라면 추천하고 싶은 다양한 창업아이템 무료창업설명회 일정을 안내드립니다.
(광고) [멸치쇼핑] 온라인 쇼핑은 멸치쇼핑에서 빠르고 쉽게~!
(광고) [멸치쇼핑] 온라인 쇼핑은 멸치쇼핑에서 빠르고 쉽게~!
[무료창업설명회]창업아이템으로 이런 건 어떠세요?
(광고) [멸치쇼핑] 온라인 쇼핑은 멸치쇼핑에서 빠르고 쉽게~!


# browser -> download 함수 이용해서 만들기

In [ ]:
browser.get("https://mail.naver.com/")
browser.page_source

In [89]:
html = getDownload("https://mail.naver.com/")
# html.text


# 로그인 하라고 나올 것임 그래서 쿠키 값을 활용하여 가져와 볼 것임

In [98]:
from requests import Session

session = Session()

In [101]:
for cookie in browser.get_cookies():
    print(cookie["name"], cookie["value"])
    session.cookies.set(cookie["name"], cookie["value"])

NID_JKL f5Kuuv2MF9bRP7XEcQS8uDEEEnGGR+mir/erimX8ets=
NNB QLMQMH6SCLKFY
NMUSER udblKqEwaqbsFAvmFxEXbrk9W4+cWqEdFonwFxEXFAgZKqMdaqEwKogsKAnsadEsHoKma9vsxonOaxRVadUstoRVaqRVadMsarRTBdRLa9vstonsH405pzk/7xE5W4d5W4JrpBU5MreR7A2lKAgs16lvpB2RFL9qp6FTW43CbNv5W4d5W4JrpBU5MreR7A2lKAgs
nid_inf 1745108946
NID_AUT 6UNNMY+A8PZ2JV4lzl6DVZLro+WFEuv664UD01iZVrhejG9n6Pp6huw3ofnr44UT
NID_SES AAABi1vn2zVuW4QaCK/StOeVHhxCP0m0FoeY1T3IXr1G6faCO2CJqeTZ78SXXidjHultznNZeT2dTkuR1kKanB/Jvxi2eA/JgDmSByvUYYFsJHjAKzzkEslsK6uFFAOLHlLw5qGcb5dkAndGAe+Oh8EM4Qa0gQf4GpFUe+vVU3HY9uz2UzM0+MY7IiWD8+uQu/stIwxenYoYPJZ2BdflbYoNA30vAqDlsgxg6aXL8MNvpMAT56twjwJCNfg7Or1s0cokQ4fOmRcjwZDJOIO5oPB/oCQ1IYgSjxIjfAju66rBeYZLY23vMiJfzTR8SPgnjtk5UfwBe9OB6S9J9DZlaU8HCKWMadrHbXjgMtoWdIIaUcs/hEr3KbHPPUnSI4rqlTy0kbpNGFxaFB1PQKqrrblTC2IhVRHOop5ufZMMMjfmrRSsYr9wdOptyzUCXne2iokGHMZ4BPUYn3GbiseAhmMDjELGscRyY0Jd2utSuRtdY/Kxx20dTIc25hl93nAB4nTcwnKYu+REk9Jq8J3uWsYBE7Y=


In [ ]:
html = session.post("https://mail.naver.com/#%7B%22fClass%22%3A%22list%22%2C%22oParameter%22%3A%7B%22page%22%3A1%2C%22sortField%22%3A1%2C%22sortType%22%3A0%2C%22folderSN%22%3A%22-1%22%2C%22type%22%3A%22all%22%2C%22isUnread%22%3Afalse%7D%7D")
# html.text

In [83]:
browser.get_cookies()

[{'domain': '.naver.com',
  'httpOnly': False,
  'name': 'NID_JKL',
  'path': '/',
  'secure': True,
  'value': 'f5Kuuv2MF9bRP7XEcQS8uDEEEnGGR+mir/erimX8ets='},
 {'domain': '.naver.com',
  'expiry': 2524640400.283537,
  'httpOnly': False,
  'name': 'NNB',
  'path': '/',
  'secure': False,
  'value': 'QLMQMH6SCLKFY'},
 {'domain': 'mail.naver.com',
  'httpOnly': False,
  'name': 'NMUSER',
  'path': '/',
  'secure': False,
  'value': 'udblKqEwaqbsFAvmFxEXbrk9W4+cWqEdFonwFxEXFAgZKqMdaqEwKogsKAnsadEsHoKma9vsxonOaxRVadUstoRVaqRVadMsarRTBdRLa9vstonsH405pzk/7xE5W4d5W4JrpBU5MreR7A2lKAgs16lvpB2RFL9qp6FTW43CbNv5W4d5W4JrpBU5MreR7A2lKAgs'},
 {'domain': '.naver.com',
  'httpOnly': False,
  'name': 'nid_inf',
  'path': '/',
  'secure': False,
  'value': '1745108946'},
 {'domain': '.naver.com',
  'httpOnly': True,
  'name': 'NID_AUT',
  'path': '/',
  'secure': False,
  'value': '6UNNMY+A8PZ2JV4lzl6DVZLro+WFEuv664UD01iZVrhejG9n6Pp6huw3ofnr44UT'},
 {'domain': '.naver.com',
  'httpOnly': False,
  'name'

# 다음 로그인
### 다음같은경우는 바로 로그인이 안됨
### iframe으로 박아놈 iframe을 접근해서 로그인 해야함

In [118]:
url = "https://www.daum.net/"
browser.get(url)
iframe = browser.find_element_by_css_selector('#loginForm')

### dom 변경

In [125]:
browser.switch_to_frame(iframe)

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.frame instead
  """Entry point for launching an IPython kernel.


In [122]:
browser.find_element_by_id("id").send

<selenium.webdriver.remote.webelement.WebElement (session="ad0d3774bd1f85a1899427c7887bcd79", element="0.4990175647646904-1")>

In [123]:
browser.find_element_by_id("id")
browser.find_element_by_id("id").send_keys("cjsfldks1109")
browser.find_element_by_id("inputPwd").clear()
browser.find_element_by_id("inputPwd").send_keys("seung32ho^^")

In [126]:
browser.find_element_by_id("loginSubmit").click()

### 원래 DOM으로 돌아오기

In [124]:
browser.switch_to_default_content()

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.default_content instead
  """Entry point for launching an IPython kernel.


# KT 로그인 - 새창 뜨는 것

In [139]:
url = "https://www.kt.com/"
browser.get(url)

In [144]:
browser.find_element_by_xpath("//*[@id='cfmClFloating']/div/div/div[2]/span[1]/a[1]").click()

### 새창으로 switch 시키기

In [146]:
browser.switch_to_window(browser.window_handles[-1])

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.window instead
  """Entry point for launching an IPython kernel.


### 원래대로 돌아가기

In [147]:
browser.switch_to_window(browser.window_handles[0])

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.window instead
  """Entry point for launching an IPython kernel.


# 합치기
### 한꺼번에 하면 안됨
### why? 요청을 기다리지 않기 때문에
### ajax -> data요청(get) -> 브라우저가 받아서 -> DOM업데이트 -> 화면에 그림(렌더링)

In [165]:
url = "http://example.webscraping.com/places/default/search"
browser.get(url)

browser.find_element_by_xpath("//input[@id='search_term']").send_keys("a")
browser.find_element_by_id("search").click()

wait.until(lambda x:x.find_elements_by_xpath("//div[@id='results']//a"))
for _ in browser.find_elements_by_xpath("//div[@id='results']//a"):
#     print(_.tag_name)
    print(_.text)
    print(_.get_attribute("href"))

Afghanistan
http://example.webscraping.com/places/default/view/Afghanistan-1
Aland Islands
http://example.webscraping.com/places/default/view/Aland-Islands-2
Albania
http://example.webscraping.com/places/default/view/Albania-3
Algeria
http://example.webscraping.com/places/default/view/Algeria-4
American Samoa
http://example.webscraping.com/places/default/view/American-Samoa-5
Andorra
http://example.webscraping.com/places/default/view/Andorra-6
Angola
http://example.webscraping.com/places/default/view/Angola-7
Anguilla
http://example.webscraping.com/places/default/view/Anguilla-8
Antarctica
http://example.webscraping.com/places/default/view/Antarctica-9
Antigua and Barbuda
http://example.webscraping.com/places/default/view/Antigua-and-Barbuda-10


In [150]:
from selenium.common.exceptions import NoSuchElementException
wait = webdriver.support.ui.WebDriverWait(browser, 10, 0.5, NoSuchElementException)

## 정확히 몇초 기다리는 함수!

In [151]:
browser.implicitly_wait(1)

# 내일 할 프로젝트
프로젝트 => 검색엔진(데이터), 문서분류, 스팸분류, 감성분석<br>
for 검색엔진(데이터), 문서분류<br>
다음 네이트 뉴스 <br>
경로, 제목 ,요약<br>
-> 파일경로, 기사제목, 내용<br>
파일경로 -> 교유번호 + 카테고리(6)<br>

정치-몇번째뉴스.txt 저장 ... 240개 문서<br>

NLP 테크닉을 이용한 전처리를 위한 데이터 모으기 =? TOKENIZATION, MA, POS, NGRAM, BPE=> STEMMING, RE